In [1]:
# data parsing because its an image
# self tip: use yandex ocr -> text

import pandas as pd

golf_df = pd.DataFrame()

#add outlook
golf_df['Outlook'] = ['rainy', 'rainy', 'overcast', 'sunny', 'sunny', 'sunny', 
                     'overcast', 'rainy', 'rainy', 'sunny', 'rainy', 'overcast',
                     'overcast', 'sunny']

#add temperature
golf_df['Temperature'] = ['hot', 'hot', 'hot', 'mild', 'cool', 'cool', 'cool',
                         'mild', 'cool', 'mild', 'mild', 'mild', 'hot', 'mild']

#add humidity
golf_df['Humidity'] = ['high', 'high', 'high', 'high', 'normal', 'normal', 'normal',
                      'high', 'normal', 'normal', 'normal', 'high', 'normal', 'high']

#add windy
golf_df['Windy'] = ['false', 'true', 'false', 'false', 'false', 'true', 'true',
                   'false', 'false', 'false', 'true', 'true', 'false', 'true']

#finally add play
golf_df['Play'] = ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 
                  'yes', 'yes', 'no']

In [2]:
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
mytree = tree.DecisionTreeClassifier()
label_encoder = LabelEncoder()
for col in golf_df.columns:
    encoded_col = label_encoder.fit_transform(golf_df[col])
    print(col + " :")
    for label, value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
        print(label, '-', value)
    golf_df[col] = encoded_col
    
print(golf_df)
# golf_df = pd.get_dummies(golf_df)
# mytree.fit(golf_df, golf_df['Play'])

Outlook :
overcast - 0
rainy - 1
sunny - 2
Temperature :
cool - 0
hot - 1
mild - 2
Humidity :
high - 0
normal - 1
Windy :
false - 0
true - 1
Play :
no - 0
yes - 1
    Outlook  Temperature  Humidity  Windy  Play
0         1            1         0      0     0
1         1            1         0      1     0
2         0            1         0      0     1
3         2            2         0      0     1
4         2            0         1      0     1
5         2            0         1      1     0
6         0            0         1      1     1
7         1            2         0      0     0
8         1            0         1      0     1
9         2            2         1      0     1
10        1            2         1      1     1
11        0            2         0      1     1
12        0            1         1      0     1
13        2            2         0      1     0


In [3]:
mytree.fit(golf_df[['Outlook', 'Temperature', 'Humidity', 'Windy']], golf_df['Play'])

DecisionTreeClassifier()

In [4]:
# to predict for rainy, cool, high, true
print(mytree.predict([[1, 0, 0, 1]])) # prints 0 - no

# to predict for sunny, mild, normal, false
print(mytree.predict([[2, 2, 1, 0]])) # prints 1 - yes

# disregard warning, its because the input is not a dataframe with valid feature names
# but just the 'row' andit still works but error

[0]
[1]


c:\Users\advai\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\advai\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [5]:
import nltk
from nltk.corpus import gutenberg

for file_id in gutenberg.fileids():
    print(file_id)

austen-emma.txt
austen-persuasion.txt
austen-sense.txt
bible-kjv.txt
blake-poems.txt
bryant-stories.txt
burgess-busterbrown.txt
carroll-alice.txt
chesterton-ball.txt
chesterton-brown.txt
chesterton-thursday.txt
edgeworth-parents.txt
melville-moby_dick.txt
milton-paradise.txt
shakespeare-caesar.txt
shakespeare-hamlet.txt
shakespeare-macbeth.txt
whitman-leaves.txt


In [6]:
# Load book data from Gutenberg library
books = {}
for file_id in gutenberg.fileids():
    books[file_id] = gutenberg.raw(file_id)
    # print(file_id)

# Preprocess the text data
# runtime ~ 50s
for book_id, text in books.items():
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Stem the tokens
    porter = nltk.PorterStemmer()
    stemmed_tokens = [porter.stem(token) for token in filtered_tokens]
    
    # Store the preprocessed text back in the dictionary
    books[book_id] = ' '.join(stemmed_tokens)
    

In [7]:
genres = {'austen-emma.txt': 'romance', 'austen-persuasion.txt': 'romance', 'austen-sense.txt': 'romance', 
          'bible-kjv.txt': 'religion', 'blake-poems.txt': 'poetry', 'bryant-stories.txt': 'fiction', 
          'burgess-busterbrown.txt': 'children', 'carroll-alice.txt': 'children', 'chesterton-ball.txt': 'mystery', 
          'chesterton-brown.txt': 'mystery', 'chesterton-thursday.txt': 'mystery', 'edgeworth-parents.txt': 'fiction', 
          'melville-moby_dick.txt': 'adventure', 'milton-paradise.txt': 'poetry', 'shakespeare-caesar.txt': 'tragedy',  
          'shakespeare-hamlet.txt' : 'tragedy', 'shakespeare-macbeth.txt' : 'tragedy', 'whitman-leaves.txt' : 'poetry'}


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Step 1: Create a list of preprocessed book texts and a list of their corresponding genres
book_texts = []
book_genres = []
book_names = []
for book_id, text in books.items():
    # Check if the book has a genre
    if book_id in genres:
        # Append the preprocessed text and its genre to their respective lists
        book_texts.append(text)
        book_genres.append(genres[book_id])
        book_names.append(book_id)
    else:
        # Add a default genre if the book doesn't have one
        book_texts.append(text)
        book_genres.append('unknown')
        book_names.append(book_id)

encoded_col = label_encoder.fit_transform(book_genres)
print("genre encodings :")
for label, value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(label, '-', value)
book_genres = encoded_col

# Step 2: Convert the preprocessed text into a bag of words representation
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(book_texts)

x_train, x_test, y_train, y_test = train_test_split(
    X, book_genres, test_size = 0.2, random_state = 42
)

# Step 3: Train a decision tree classifier on the bag of words representation and their corresponding genres
clf = tree.DecisionTreeClassifier()
clf.fit(x_train, y_train)

genre encodings :
adventure - 0
children - 1
fiction - 2
mystery - 3
poetry - 4
religion - 5
romance - 6
tragedy - 7


DecisionTreeClassifier()

In [10]:
predictions = clf.predict(x_test)

# Print the predicted and actual genres for the test data
predicted_genres = label_encoder.inverse_transform(predictions)
actual_genres = label_encoder.inverse_transform(y_test)
book_names = [book_id for i, book_id in enumerate(books) if i in x_test.indices]

print("Name of book\t| Actual Genre\t| Predicted Genre")
print("--------------------------------------------------")
for book, actual, predicted in zip(book_names, actual_genres, predicted_genres):
    print("{:<15}\t| {:<15}\t| {:<15}".format(book, actual, predicted))

Name of book	| Actual Genre	| Predicted Genre
--------------------------------------------------
austen-emma.txt	| romance        	| romance        
austen-persuasion.txt	| romance        	| children       
bryant-stories.txt	| mystery        	| children       
burgess-busterbrown.txt	| fiction        	| children       
